In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

DB_NAME = "habits.db"
CSV_NAME = "habit_log.csv"

def connect():
    return sqlite3.connect(DB_NAME)

def create_table():
    conn = connect()
    cursor = conn.cursor()

    cursor.execute("""
        CREATE TABLE IF NOT EXISTS habit_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT,
            social_media REAL DEFAULT 0
        );
    """)

    conn.commit()
    conn.close()

def add_habit():
    habit = input("Enter habit name (no spaces): ").strip().replace(" ", "_")

    conn = connect()
    cursor = conn.cursor()

    try:
        cursor.execute(f"ALTER TABLE habit_log ADD COLUMN {habit} REAL DEFAULT 0;")
        conn.commit()
        print(f"✔ Habit '{habit}' added!")
    except sqlite3.OperationalError:
        print("⚠ Habit already exists or invalid name.")

    conn.close()
    update_csv()

def add_entry():
    conn = connect()
    cursor = conn.cursor()

    date_str = input("Enter date (YYYY-MM-DD): ").strip()
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
    except ValueError:
        print("❗ Invalid date format.")
        return

    cursor.execute("SELECT 1 FROM habit_log WHERE date=?", (date_str,))
    if cursor.fetchone():
        print(f"❗ Entry for {date_str} already exists. One entry per day allowed.")
        conn.close()
        return

    cursor.execute("PRAGMA table_info(habit_log);")
    columns = [col[1] for col in cursor.fetchall() if col[1] != "id"]

    values = {}
    print("\nEnter hours for each:")
    for col in columns:
        if col == "date":
            values[col] = date_str
            continue
        val = input(f"{col}: ").strip()
        values[col] = float(val) if val else 0

    col_names = ", ".join(values.keys())
    placeholders = ", ".join(["?"] * len(values))
    cursor.execute(f"INSERT INTO habit_log ({col_names}) VALUES ({placeholders})", list(values.values()))

    conn.commit()
    conn.close()
    update_csv()

    print("✔ Entry added successfully!")

def view_table():
    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log ORDER BY date", conn)
    conn.close()

    if df.empty:
        print("⚠ No entries found.")
    else:
        print("\n======== HABIT LOG =========")
        print(df)

def update_csv():
    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log ORDER BY date", conn)
    conn.close()
    df.to_csv(CSV_NAME, index=False)

def calculate_streaks():
    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log ORDER BY date", conn)
    conn.close()

    if df.empty:
        print("⚠ No data for streaks.")
        return

    df["date"] = pd.to_datetime(df["date"])

    streaks = {}
    habit_cols = [c for c in df.columns if c not in ("id", "date")]

    for habit in habit_cols:
        habit_df = df[["date", habit]].sort_values("date", ascending=False)
        streak = 0
        expected = habit_df.iloc[0]["date"]

        for _, row in habit_df.iterrows():
            if row["date"] == expected and row[habit] > 0:
                streak += 1
                expected -= timedelta(days=1)
            else:
                break

        streaks[habit] = streak

    print("\n======== STREAKS =========")
    for habit, s in streaks.items():
        print(f"{habit}: {s} days")

def pie_chart_stats():
    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log", conn)
    conn.close()

    if df.empty:
        print("⚠ No data available for stats.")
        return

    habit_cols = [c for c in df.columns if c not in ("id", "date")]

    totals = df[habit_cols].sum()

    plt.figure(figsize=(7, 7))
    plt.pie(totals, labels=totals.index, autopct="%1.1f%%")
    plt.title("Total Time Breakdown Across Habits")
    plt.show(block=False)
    plt.pause(0.1)


def bar_chart_for_habit():
    habit = input("Enter habit name: ").strip().replace(" ", "_")

    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log ORDER BY date", conn)
    conn.close()

    if habit not in df.columns:
        print("⚠ Habit does not exist in database.")
        return

    df["date"] = pd.to_datetime(df["date"])

    plt.figure(figsize=(10, 5))
    sns.barplot(x=df["date"], y=df[habit])
    plt.xticks(rotation=45)
    plt.ylabel("Hours")
    plt.title(f"Time Spent on {habit} Over Time")
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.1)


def social_media_vs_habits():
    conn = connect()
    df = pd.read_sql_query("SELECT * FROM habit_log", conn)
    conn.close()

    if df.empty:
        print("⚠ Not enough data.")
        return

    df["date"] = pd.to_datetime(df["date"])

    habit_cols = [c for c in df.columns if c not in ("id", "date", "social_media")]
    df['total_habit_time'] = df[habit_cols].sum(axis=1)

    plt.figure(figsize=(12,6))
    plt.bar(df['date'], df['total_habit_time'], label='Total Habit Time', color='tab:green')
    plt.bar(df['date'], df['social_media'], label='Social Media', color='tab:red', alpha=0.7)
    plt.title("Daily Habit Time vs Social Media Usage")
    plt.xlabel("Date")
    plt.ylabel("Hours")
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.1)



def main():
    create_table()

    while True:
        print("\n===== HABIT TRACKER MENU =====")
        print("1. Add new habit")
        print("2. Add new entry")
        print("3. View full log")
        print("4. View streaks")
        print("5. Pie chart of habits")
        print("6. Bar graph for a specific habit")
        print("7. Social media vs all habits")
        print("8. Exit")

        choice = input("Choose: ").strip()

        if choice == "1":
            add_habit()
        elif choice == "2":
            add_entry()
        elif choice == "3":
            view_table()
        elif choice == "4":
            calculate_streaks()
        elif choice == "5":
            pie_chart_stats()
        elif choice == "6":
            bar_chart_for_habit()
        elif choice == "7":
            social_media_vs_habits()
        elif choice == "8":
            print("Goodbye!")
            break
        else:
            print("Invalid choice!")


if __name__ == "__main__":
    main()
